In [ ]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from time import sleep
from pathlib import Path
import os

Format all the rows in the textfile so that the urls are correct

In [ ]:
df = pd.read_csv("wiktionary_urls.txt",sep="\t",header=None)

In [ ]:
#df.columns = ["unclean_url"]

In [ ]:
def format_text(row):
    if not row.startswith("http"):
        return row.split(" ")[1]
    else:
        return row

In [ ]:
#df["clean_url"] = df["unclean_url"].apply(lambda row: format_text(row))

In [ ]:
index = list(range(2400,2700))

Download the pages

In [ ]:
PATH_TO_TARGET_FOLDER = "downloads/"

In [ ]:
def download(url,target_path, filename):
    urllib.request.urlretrieve(url, target_path + filename)
    sleep(0.1)
    
def downloader(df, starting_index, max_index, target_path):
    
    current_starter = starting_index
    current_end = current_starter + 300
    
    while current_end <= max_index:
    
        index = list(range(current_starter,current_end))
    
        df.iloc[index].apply(
        lambda row: download(row["clean_url"],target_path,
                         "document_{num}.html".format(num=row.name))
        ,axis=1)
        
        sleep(5)
        print("Done with download from {start} to {end}".format(start=current_starter, end=current_end))
        
        current_starter = current_starter + 300
        current_end = current_starter + 300
        
        

In [ ]:
downloader(df, 13800, 14100,PATH_TO_TARGET_FOLDER)

In [ ]:
index = list(range(14100,14109))

In [ ]:
df

In [ ]:
df.iloc[index]

Extract information from the pages

In [ ]:
def get_information(filename):
    with open(filename) as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        
    tables = soup.find_all("table", class_="inflection-table")
    table = ""
    
    if len(tables) > 0:
        table = tables[0]
    else:
        print("Does not have any table")
        print(filename)
        return ("-","-","-")
    
    tds = table.find_all("td")
    
    texts = []

    for td in tds:
        splitted = td.text.split("\n")
        
        if len(splitted) > 2:
            texts.append(splitted[0] + "_" + splitted[2])
        else:
            texts.append(splitted[0])
    
    return tuple(texts)

In [ ]:
get_information("test.html")

Do it for all html documents

In [ ]:
INPUT_FOLDER = "downloads/"
TARGET_FOLDER = "downloads/done/"

In [ ]:
# Load all files
files = Path(INPUT_FOLDER).glob('*.html')

In [ ]:
data = []

In [ ]:
len(data)

In [ ]:
for f in files:
    information = get_information(f)
    data.append(information)
    current_filename = f.stem
    os.rename(f, TARGET_FOLDER + current_filename + ".html")

Turn into resulting dataframe

In [ ]:
for f in to_repair_files:
    current_filename = f.stem.split("done")[1]
    os.rename(f, INPUT_FOLDER + current_filename + ".html")

In [ ]:
faulty_data = []

In [ ]:
len(faulty_data)

In [ ]:
index = 0
while index < len(data):
    if len(data[index]) > 3:
        faulty_data.append(data[index])
        data.pop(index)
    index = index + 1

In [ ]:
for dp in data:
    if len(dp) > 3:
        print(dp)

In [ ]:
df = pd.DataFrame(data, columns=['positiv', 'komparativ', 'superlativ'])

In [ ]:
df.to_csv("nearly_all.csv",index=False)

In [ ]:
df

In [ ]:
def words_have_same_stem(row):
    pos = row["positiv"]
    komp = row["komparativ"]
    sup = row["superlativ"]
    
    if pos in komp and pos in sup:
        return True
    else:
        return False

In [ ]:
df["have_same_stem"] = df.apply(lambda row: words_have_same_stem(row), axis=1)

In [ ]:
# object where isna() is true need to be dropped
df_irregular = df[~df["have_same_stem"] & (df["komparativ"] != "—") & (df["superlativ"] != "—")]

In [ ]:
df_irregular.to_csv("adjektive_irreguläre_steigerung.csv",index=False)

In [ ]:
#set(df_irregular["komparativ"].to_list())

In [ ]:
"besser" in set(df["komparativ"])